In [ ]:
#Installing the libraray for Data Visualisation for LDA

!pip install pyLDAvis 

In [ ]:
#For english letter parsing, follow the same step as above. Open commnad prompt with admin rights and run the below command
##      python -m spacy download en 

In [ ]:
#Loading English language

from spacy.lang.en import English

In [ ]:
#Creeating an object of English class to parse the data

parser = English()

In [ ]:
#Writing functions for data cleaning
# Starting from tokenization

def tokenize(text):
    lda_tokens = []
    #tokens = text
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url: #Does the token resemples URL
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'): #Does token starts with @
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [ ]:
#Importing nltk. Its a heavy library so might take time. We need only wordnet part of NLTK not complete package
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
#Functions for Lemmma

from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

In [ ]:
word = 'Happpyness'
def get_lemma(word):
    lemma = wn.morphy(word, 'n')
    if lemma is None:
        return word
    else:
        return lemma
get_lemma(word)

'Happpyness'

In [ ]:
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)
get_lemma(word)

'Happpyness'

In [ ]:
#Accessing only the stop words and creating an object of it

nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#Prepraring data for LDA

def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [ ]:
#Accessing dataset. 
text_data = []
with open('/content/drive/MyDrive/NLP IMS/Day5/day 5/dataset.csv') as f:
    for line in f:
        tokens = prepare_text_for_lda(line)
        print(tokens)
        text_data.append(tokens)
        

#LDA with Gensim

In [ ]:
#Importing gensim. If this gives error, download gensim using pip command 
# !pip install gensim 

from gensim import corpora

In [ ]:
#Accessing dictionary object and making corpus
#bag of words, keeps a count of the total occurrences of most frequently used words.

#In order to work on text documents, Gensim requires the words (aka tokens) 
#be converted to unique ids. In order to achieve that, 
#Gensim lets you create a Dictionary object that maps each word to a unique id.

# Corpus is typically a ‘collection of documents as a bag of words’.
dictionary = corpora.Dictionary(text_data)

corpus = [dictionary.doc2bow(text) for text in text_data]

import pickle #Python object hierarchy is converted into a byte stream, 
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [ ]:
#Applying LDA

import gensim
NUM_TOPICS = 5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model5.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.015*"based" + 0.013*"large" + 0.012*"video" + 0.012*"image"')
(1, '0.020*"network" + 0.018*"wireless" + 0.016*"sensor" + 0.015*"based"')
(2, '0.022*"system" + 0.020*"design" + 0.017*"based" + 0.011*"services"')
(3, '0.038*"network" + 0.012*"wireless" + 0.010*"mobile" + 0.009*"system"')
(4, '0.025*"network" + 0.024*"based" + 0.018*"query" + 0.016*"using"')


In [ ]:
#Testing LDA model by giving input string

new_doc = 'Influence of external input on Oscillatory Cellular Nonlinear Networks dynamics.'


new_doc = prepare_text_for_lda(new_doc)
new_doc_bow = dictionary.doc2bow(new_doc)
print(new_doc_bow)
print(ldamodel.get_document_topics(new_doc_bow))

[(24, 1), (82, 1), (201, 1), (357, 1), (721, 1), (722, 1), (723, 1), (724, 1)]
[(0, 0.022228414), (1, 0.023094289), (2, 0.022377672), (3, 0.022619208), (4, 0.90968037)]


In [ ]:
#Creating visualisation of the LDA model

dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model5.gensim')
import pyLDAvis.gensim_models
lda_display = pyLDAvis.gensim_models.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_lda.py:29: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).eps


----------------------------------